# 07. Dense Network

In [3]:
import numpy as np
import tensorflow.keras as keras

First we load the data

In [4]:
with np.load("train_data.npz",allow_pickle=False) as train_npz:
    train_data = dict(train_npz.items())

with np.load("valid_data.npz",allow_pickle=False) as valid_npz:
    valid_data = dict(valid_npz.items())

with np.load("test_data.npz",allow_pickle=False) as test_npz:
    test_data = dict(test_npz.items())

In [5]:
X_train,y_train = train_data.values()
X_valid,y_valid = valid_data.values()
X_test,y_test = test_data.values()

Let's concatenate our train and validation data to use this in our fit.

In [6]:
X_train = np.concatenate((X_train,X_valid))
y_train = np.concatenate((y_train,y_valid))

Let's create our model with keras and make one dense layer to get the 6 outputs we want, I use softmax activation as it's the last layer.

In [7]:
X_train.shape

(420, 2048)

In [5]:
model1d = keras.Sequential()
model1d.add(keras.layers.Dense(units=6,activation="softmax",input_dim=2048))
model1d.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 6)                 12294     
Total params: 12,294
Trainable params: 12,294
Non-trainable params: 0
_________________________________________________________________


We see that we get the expected number of parameters : 2048\*6+6 = 12294.

Let's now define the training elements and an early stopping to 5.

In [6]:
model1d.compile(optimizer="adam",loss="sparse_categorical_crossentropy",metrics=["acc"])
early_stopping = keras.callbacks.EarlyStopping(patience=5)

Let's now fit our model

In [7]:
history1d = model1d.fit(X_train,y_train,batch_size=32,epochs=100,validation_split=0.25,callbacks=[early_stopping])

Train on 315 samples, validate on 105 samples
Epoch 1/100
315/315 [==============================] - 1s 2ms/step - loss: 1.5419 - acc: 0.4063 - val_loss: 0.9797 - val_acc: 0.8095
Epoch 2/100
315/315 [==============================] - 0s 113us/step - loss: 0.7357 - acc: 0.8730 - val_loss: 0.5036 - val_acc: 0.9429
Epoch 3/100
315/315 [==============================] - 0s 120us/step - loss: 0.4042 - acc: 0.9238 - val_loss: 0.3321 - val_acc: 0.9429
Epoch 4/100
315/315 [==============================] - 0s 130us/step - loss: 0.2884 - acc: 0.9397 - val_loss: 0.2704 - val_acc: 0.9429
Epoch 5/100
315/315 [==============================] - 0s 109us/step - loss: 0.2241 - acc: 0.9556 - val_loss: 0.2478 - val_acc: 0.9333
Epoch 6/100
315/315 [==============================] - 0s 136us/step - loss: 0.1892 - acc: 0.9683 - val_loss: 0.2243 - val_acc: 0.9429
Epoch 7/100
315/315 [==============================] - 0s 99us/step - loss: 0.1671 - acc: 0.9714 - val_loss: 0.2144 - val_acc: 0.9333
Epoch 8/100


In [8]:
mean_val_acc1d = np.mean(history1d.history["val_acc"][-3:])
print("Mean validation accuracy (last 3) : {:.1f}%".format(100*mean_val_acc1d))

Mean validation accuracy (last 3) : 92.4%


Let's now add a hidden layer with a ReLU activation and 10 outputs to our model

In [9]:
model2d = keras.Sequential()
model2d.add(keras.layers.Dense(units=10,activation="relu",input_dim=2048))
model2d.add(keras.layers.Dense(units=6,activation="softmax"))
model2d.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 10)                20490     
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 66        
Total params: 20,556
Trainable params: 20,556
Non-trainable params: 0
_________________________________________________________________


In [10]:
model2d.compile(optimizer="adam",loss="sparse_categorical_crossentropy",metrics=["acc"])

In [11]:
history2d = model2d.fit(X_train,y_train,batch_size=32,epochs=100,validation_split=0.25,callbacks=[early_stopping])

Train on 315 samples, validate on 105 samples
Epoch 1/100
315/315 [==============================] - 1s 2ms/step - loss: 1.4716 - acc: 0.4794 - val_loss: 1.0288 - val_acc: 0.6857
Epoch 2/100
315/315 [==============================] - 0s 117us/step - loss: 0.8413 - acc: 0.7714 - val_loss: 0.6586 - val_acc: 0.8476
Epoch 3/100
315/315 [==============================] - 0s 123us/step - loss: 0.5810 - acc: 0.8540 - val_loss: 0.4954 - val_acc: 0.8762
Epoch 4/100
315/315 [==============================] - 0s 106us/step - loss: 0.4217 - acc: 0.8667 - val_loss: 0.3996 - val_acc: 0.8857
Epoch 5/100
315/315 [==============================] - 0s 104us/step - loss: 0.3302 - acc: 0.9270 - val_loss: 0.3449 - val_acc: 0.9048
Epoch 6/100
315/315 [==============================] - 0s 107us/step - loss: 0.2645 - acc: 0.9397 - val_loss: 0.3053 - val_acc: 0.9238
Epoch 7/100
315/315 [==============================] - 0s 105us/step - loss: 0.2259 - acc: 0.9651 - val_loss: 0.2845 - val_acc: 0.9048
Epoch 8/100

In [12]:
mean_val_acc2d = np.mean(history2d.history["val_acc"][-3:])
print("Mean validation accuracy (last 3) : {:.1f}%".format(100*mean_val_acc2d))

Mean validation accuracy (last 3) : 93.0%


We get similar results in accuracy with 2 layers

Let's compute and collect our test accuracies :

In [13]:
(nn1_test_loss,nn1_test_acc) = model1d.evaluate(X_test,y_test)
(nn2_test_loss,nn2_test_acc) = model2d.evaluate(X_test,y_test)

50/50 [==============================] - 0s 129us/step


In [14]:
nn1_test_acc = nn1_test_acc*100
nn2_test_acc = nn2_test_acc*100

In [15]:
print("Test accuracy nn1 : {:.1f}%".format(nn1_test_acc))
print("Test accuracy nn2 : {:.1f}%".format(nn2_test_acc))

Test accuracy nn1 : 96.0%
Test accuracy nn2 : 96.0%


In [16]:
%store nn1_test_acc
%store nn2_test_acc

Stored 'nn1_test_acc' (float64)
Stored 'nn2_test_acc' (float64)
